In [54]:
#Import some libraries
import numpy as np
import pandas as pd
# from thermoModel import *
from generateThermocline import generateThermocline
from maxDepth import maxDepth
from julianToNormal import jd_to_date
import matplotlib.pyplot as plt
from Net import Net
from global_land_mask import globe
import os
import xarray
import torch
import geopandas as gpd
import geodatasets

In [64]:
# setup global variables
latRange = [-90,90]                # range of world latitudes (degrees)
longRange = [-180,180]             # range of world longitudes (degrees)
dateRange = [2455562.5, 2455927.5]  # range of dates (julian time/days)

areaIncr = 5        # world area grid (degrees)
depthIncr = 1       # depth increment (meter)
dateIncr = 500        # date increment (days)
tempCutoff = 1      # thermocline temperature cutoff (K or degC)

latArray = np.arange(latRange[0], latRange[1], areaIncr)
longArray = np.arange(longRange[0], longRange[1], areaIncr)
dateArray = np.arange(dateRange[0], dateRange[1], dateIncr)

In [8]:
def generateThermoclineActual(df, lat, long, thermoActualPath):
    err = 0.1
    filtered_df = df.loc[(df['Latitude'].between(lat - err, lat + err)) 
            & (df['Longitude'].between(long - err, long + err)), ['Depth', 'Temperature']]
    # not enough data to filter by time as well
    sorted_df = filtered_df.sort_values(by=['Depth'])
    
    fig, ax = plt.subplots()
    ax.plot(sorted_df["Temperature"], sorted_df["Depth"])  
    ax.set_xlabel('Temperature ($^\circ$C)')
    ax.set_ylabel('Depth (m)')
    ax.set_title(f'Thermocline Actual for Lat: {lat:,.1f} Long: {long:,.1f}')
    ax.invert_yaxis()
    plt.savefig(f'{thermoActualPath}/ThermoclineActual_{lat:,.1f}_{long:,.1f}.png')
    plt.close('all')

    return sorted_df

In [9]:
def generateSurfaceTempActualMap(plotDf, surfaceTempActualPath, date):
    plotColor = "lightgrey"
    opacity = 0.3
    worldmap = gpd.read_file(geodatasets.get_path("naturalearth.land"))

    # Creating axes and plotting world map
    fig, ax = plt.subplots(figsize=(12, 6))
    worldmap.plot(color=plotColor, ax=ax)

    # Plotting our Impact Energy data with a color map
    x = plotDf['Longitude']
    y = plotDf['Latitude']
    z = plotDf['Surface_Temp_Actual']
    plt.scatter(x, y, c=z, alpha=opacity, marker=".")
    plt.colorbar(label='Actual Surface Temp ($^\circ$C)')
    plt.title("Global Actual Surface Temperature Map")
    plt.xlabel("Longitude ($^\circ$)")
    plt.ylabel("Latitude ($^\circ$)")

    year,month,day = jd_to_date(date)
    plt.savefig(f'{surfaceTempActualPath}/SurfaceTempActual_{int(month)}-{int(day)}-{str(int(year))[2:]}.png')
    plt.close('all')


In [50]:
# import training dataset for depth checking
print("Importing training data....")
train_dataset = "practiceData3Years.nc"
ds = xarray.open_dataset(train_dataset)
df = ds.to_dataframe()

# create results directories if it doesn't exist
print("Creating directories....")
resultsPath = "ResultsTest"
thermoPath = resultsPath + "/ThermoclinePlots"
thermoActualPath = resultsPath + "/ThermoclineActualPlots"
surfaceTempActualPath = resultsPath + "/SurfaceTempActualMaps"
if not os.path.exists(resultsPath):
    os.makedirs(resultsPath)
    os.makedirs(thermoPath)
    os.makedirs(thermoActualPath)
    os.makedirs(surfaceTempActualPath)
    print("Directories Created Successfully")

# import ML model
print("Importing model....")
modelPath = "OTEC_miniBatchState.pth"
net = Net()
net.load_state_dict(torch.load(modelPath))
net.eval()

Importing training data....
Creating directories....
Importing model....


Net(
  (fc1): Linear(in_features=4, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=100, bias=True)
  (fc4): Linear(in_features=100, out_features=100, bias=True)
  (fc5): Linear(in_features=100, out_features=100, bias=True)
  (fc6): Linear(in_features=100, out_features=1, bias=True)
)

In [88]:
# has to be defined here doesn't work if imported for some reason
def minDepthTemperature(df, lat, long, areaIncr):
    i = 0.5
    err = areaIncr*i
    latBetween = df['Latitude'].between(lat - err, lat + err)
    longBetween = df['Longitude'].between(long - err, long + err)
    filtered_df = df.loc[latBetween & longBetween,['Depth', 'Temperature']]
    if filtered_df.empty:
        return -1
    sorted_df = filtered_df.sort_values(by=['Depth'])
    print(len(sorted_df))

    return float(sorted_df['Temperature'].iloc[0])

In [77]:
minDepthTemperature(df, -80, 165, 1)

-1

In [90]:
# TODO CHECK IF SURFACE TEMP MAP GENERATION WORKS AND IF VALUES ARE RIGHT**

# main for loop to generate thermocline and get exergy for each time
print("Entering main loop....") 
for date in dateArray:
    # generate new dataframe for storing plotting variables worldwide for each new date
    print(f"Initializing plots dataframe for {date:.1f}....")
    plotDf = pd.DataFrame(columns = ['Latitude', 'Longitude', 'Surface_Temp_Actual'])
    
    # generate data for each location 
    print(f"Generating data for {date:.1f}....")
    for lat in latArray:
        for long in longArray:
            
            # land mask
            if not globe.is_land(lat, long):
                # get maximum depth of location from database
                # maxDep = maxDepth(df, lat, long, areaIncr) 

                # generate thermocline for location using ML model
                # print(f"Generating thermoclines for {lat:.1f}_{long:.1f}_{date:.1f}....") 
                # tempDf = generateThermocline(lat, long, date, maxDep, depthIncr, net, thermoPath, True)
                # tempDfActual = generateThermoclineActual(df, lat, long, thermoActualPath)

                # add data to plotting dataframe
                print(lat, long)
                plotDf = plotDf._append({'Latitude' : lat, 
                                        'Longitude' : long, 
                                        'Surface_Temp_Actual': minDepthTemperature(df, lat, long, areaIncr)}, ignore_index = True)
    
    generateSurfaceTempActualMap(plotDf, surfaceTempActualPath, date)

Entering main loop....
Initializing plots dataframe for 2455562.5....
Generating data for 2455562.5....
-85 -160
-85 -155
-80 -180
-80 -175
-80 -170
-80 -165
-80 -160
-80 -155
-80 -150
-80 -75
-80 -70
-80 -65
-80 -60
-80 -55
-80 -40
-80 -35
-80 -30
-80 165
100401
-80 170
-80 175
-75 -180
-75 -175
3178
-75 -170
3895
-75 -165
7204
-75 -160
13375
-75 -155
38094
-75 -150
19687
-75 -145
14944
-75 -140
7709
-75 -110
-75 -105
-75 -100
-75 -60
-75 -55
-75 -50
-75 -45
-75 -40
-75 -35
-75 -30
-75 -25
-75 -20
-75 165
83
-75 170
-75 175
-70 -180
-70 -175
-70 -170
7240
-70 -165
-70 -160
-70 -155
484
-70 -150
12525
-70 -145
-70 -140
2420
-70 -135
-70 -130
-70 -125
116227
-70 -120
29913
-70 -115
-70 -110
22684
-70 -105
4349
-70 -100
6402
-70 -95
5802
-70 -90
14483
-70 -85
8603
-70 -80
-70 -60
-70 -55
-70 -50
36
-70 -45
1789
-70 -40
13493
-70 -35
17794
-70 -30
18283
-70 -25
16818
-70 -20
21413
-70 -15
10573
-70 -10
8083
-70 -5
7572
-70 0
10856
-70 5
1932
-70 10
-70 15
-70 20
-70 25
-70 30
-70 70
-70 1

In [80]:
globe.is_land(-80, 165)

False